In [1]:
import os
import pandas as pd
import torch
from transformers import AlbertForSequenceClassification, AlbertTokenizer, Trainer, TrainingArguments
from torch.quantization import quantize_dynamic
from transformers import Trainer, TrainingArguments, RobertaForSequenceClassification, RobertaTokenizer
from datasets import load_dataset

from pathlib import Path
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
import torch
from torch.quantization import quantize_dynamic
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import pandas as pd
import numpy as np
import os
import copy
from datasets import load_dataset
import torch.nn.utils.prune as prune
from transformers import AlbertForSequenceClassification, AlbertTokenizer, Trainer, TrainingArguments


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Load data
export_dir = Path(os.getcwd())
data_path = Path(export_dir, "data")
train_file = Path(data_path, 'train_data_only_text_and_labels.csv')
eval_file = Path(data_path, 'eval_data_only_text_and_labels.csv')
test_file = Path(data_path, 'test_data_only_text_and_labels.csv')

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)
test_df = pd.read_csv(test_file)


In [3]:
train_df

,text_combined,label
0,mr benson eko bbennisadinetcomuy benson eko ch...,1
1,millicent boston helgagermanflintcochraneorg r...,1
2,jason ling vytekdemisehotmailcom _nextpart_001...,0
3,vladimir antalik ohrbzoznamsk would like purch...,0
4,lennart regebro hyiffbigmailcom wed mar 26 200...,0
...,...,...
52785,justin shore listuserneopittstateedu 116 pm 04...,0
52786,charles philip chan optgpesympaticoca aaron ku...,0
52787,isador kee catimen84swannejp dear d59ebf6a0f14...,1
52788,global risk management operations weekly opera...,0


In [4]:
!pip install --upgrade torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 31.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 15.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 26.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━

In [5]:


# Load the model from the checkpoint
checkpoint_path = "./results/albert/checkpoint-9900/"

# Load the tokenizer
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

# Load the model from the checkpoint directory
model = AlbertForSequenceClassification.from_pretrained(checkpoint_path)



/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Prune the model

In [6]:
# Define a helper function to apply global pruning
def apply_global_pruning(model, amount=0.3):
    # Loop through each module in the model and prune if it is a Linear or Conv layer
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear) or isinstance(module, torch.nn.Conv1d):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')  # Remove the reparameterization to make it permanent

# Apply global pruning to the model (adjust the amount based on your needs)
apply_global_pruning(model, amount=0.3)

### Save pruned model

In [8]:
torch.save(model, "./results/albert/fine_tune/prune/pruned_model.pth")



### Load pruned model

In [3]:
# Evaluate the pruned model using metric_fn function
# Load the pruned model
model = torch.load("./results/albert/fine_tune/prune/pruned_model.pth")
# Ensure the model is on the correct device (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/tmp/ipykernel_2334/2767523246.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("./results/albert/fine_tune/prune/pruned_model.pth")


AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

### Load original model

In [4]:
# Load the model from the checkpoint
checkpoint_path = "./results/albert/checkpoint-9900/"

# Load the tokenizer
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

# Load the model from the checkpoint directory
original_model = AlbertForSequenceClassification.from_pretrained(checkpoint_path)


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
def count_non_zero_weights(model):
    non_zero_count = 0
    for param in model.parameters():
        non_zero_count += torch.sum(param != 0).item()
    return non_zero_count

# Count non-zero weights in the original model
original_non_zero_weights = count_non_zero_weights(original_model)

# Count non-zero weights in the pruned model
pruned_non_zero_weights = count_non_zero_weights(model)

# Print results
print(f"Original model non-zero weights: {original_non_zero_weights}")
print(f"Pruned model non-zero weights: {pruned_non_zero_weights}")

# Calculate and print the reduction ratio
reduction_ratio = (original_non_zero_weights - pruned_non_zero_weights) / original_non_zero_weights
print(f"Reduction in non-zero weights: {reduction_ratio:.2%}")

Original model non-zero weights: 11685122
Pruned model non-zero weights: 9354857
Reduction in non-zero weights: 19.94%


In [8]:

def metric_fn(eval_pred):
    # Extract predictions and labels
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Convert logits to predicted class
    
    # Compute basic metrics
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    
    # Compute AUC
    if len(np.unique(labels)) > 1:  # Check if there is more than one class in labels
        auc = roc_auc_score(labels, logits[:, 1])  # Use probability of the positive class
    else:
        auc = float('nan')  # If only one class is present, AUC is not defined

    # Compute confusion matrix
    cm = confusion_matrix(labels, predictions)
    tn, fp, fn, tp = cm.ravel()
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'tp': tp,
    }


In [10]:
from transformers import Trainer, TrainingArguments, RobertaTokenizer
from datasets import Dataset
import pandas as pd
import time
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve

# Example test_df DataFrame


# Convert test_df to Dataset if it's a DataFrame
test_dataset = Dataset.from_pandas(test_df)

# Initialize tokenizer (make sure to use the correct tokenizer for your model)
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

# Define a function to tokenize the inputs
def tokenize_function(examples):
    return tokenizer(examples['text_combined'], padding='max_length', truncation=True)

# Apply the tokenizer to the dataset
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Define evaluation arguments
eval_args = TrainingArguments(
    output_dir='./results/albert/fine_tune/prune',
    per_device_eval_batch_size=50,
    logging_dir='./results/albert/fine_tune/prune',
    evaluation_strategy="epoch"  # Ensure evaluation is done at the end of each epoch
)

# Define the Trainer with the quantized model
trainer = Trainer(
    model=model,
    args=eval_args,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=metric_fn  # Uncomment if you have a metric function defined
)

# Evaluate the model on the tokenized test dataset
evaluation_results = trainer.evaluate()

# Print evaluation results
print(evaluation_results)


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 16498/16498 [00:27<00:00, 589.64 examples/s]
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


{'eval_loss': 0.1724466234445572, 'eval_model_preparation_time': 0.0008, 'eval_accuracy': 0.962783367680931, 'eval_precision': 0.9341545841055271, 'eval_recall': 0.9988343629793682, 'eval_f1': 0.9654123479044615, 'eval_auc': 0.9946599281591365, 'eval_tn': 7315, 'eval_fp': 604, 'eval_fn': 10, 'eval_tp': 8569, 'eval_runtime': 121.7761, 'eval_samples_per_second': 135.478, 'eval_steps_per_second': 2.71}


In [ ]:
total_params = sum(p.numel() for p in model.parameters())

print(f"Total number of parameters: {total_params}")

In [ ]:
def get_model_size(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) * 4 / (1024 ** 2)  # Size in MB
get_model_size(model)

In [ ]:
test= pd.read_csv(Path(data_path,'test_data_only_text_and_labels.csv'))
test.head()
X_test = test["text_combined"].tolist()
y_test = test["label"]


In [ ]:
test.shape

In [ ]:
import torch
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve
import numpy as np
import matplotlib.pyplot as plt
import time
# Function to evaluate the model on the test set in batches
def evaluate_in_batches(model, tokenizer, X_test, y_test, batch_size):
    device = torch.device("cpu")  # Change to "cuda" if GPU is available and required
    model.to(device)
    model.eval()

    # Split the test set into batches
    num_batches = len(X_test) // batch_size + (1 if len(X_test) % batch_size != 0 else 0)

    all_predictions = []
    all_probs = []
    all_true_labels = []
    start_time = time.time()
    with torch.no_grad():
        for i in range(num_batches):
            # Get the batch data
            batch_texts = X_test[i * batch_size: (i + 1) * batch_size]
            batch_labels = y_test[i * batch_size: (i + 1) * batch_size]

            # Tokenize the batch
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
            inputs = {key: value.to(device) for key, value in inputs.items()}

            # Get logits from the model
            outputs = model(**inputs)
            logits = outputs.logits.cpu()
            probs = torch.nn.functional.softmax(logits, dim=-1).cpu()

            # Get the predicted labels
            predictions = torch.argmax(probs, dim=1).cpu()

            # Store the results
            all_predictions.append(predictions.numpy())
            all_probs.append(probs[:, 1].numpy())
            all_true_labels.append(batch_labels.to_numpy())

    # Concatenate all batch results
    y_pred = np.concatenate(all_predictions)
    y_scores = np.concatenate(all_probs)
    y_true = np.concatenate(all_true_labels)
    total_time = time.time() - start_time
    # Calculate evaluation metrics
    cm = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_scores)
    evaluation_results = {
        "Confusion_Matrix": cm,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "AUC": auc,
        "Evaluation_Time": total_time
    }
    # Print results
    print("Confusion Matrix:")
    print(cm)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-Score:", f1)
    print("AUC:", auc)
    print("eval_time:", total_time)
    # ROC Curve
    with open(f'./results/roberta/fine_tune/prune/pruned_model_eval_results_cpu.txt', 'w') as f:
        f.write(f"{evaluation_results}\n\n")
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    plt.figure()
    plt.plot(fpr, tpr, label=f'ROC curve (area = {auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

    # Precision-Recall Curve
    precision_vals, recall_vals, _ = precision_recall_curve(y_true, y_scores)
    plt.figure()
    plt.plot(recall_vals, precision_vals, label='Precision-Recall curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.show()
    return cm
# Example usage
batch_size = 500  # Set the desired batch size
cm = evaluate_in_batches(model, tokenizer, X_test, y_test, batch_size)

 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

#Define the data matrix
data = cm

# Define axis labels
x_labels = ['Predicted Negative', 'Predicted Positive']
y_labels = ['Actual Negative', 'Actual Positive']

# Create the heatmap with values inside cells
ax = sns.heatmap(data, annot=True, fmt='d', cmap='coolwarm', cbar=True, 
                 xticklabels=x_labels, yticklabels=y_labels)

# Set axis labels
ax.set_xlabel('Predicted Label')
ax.set_ylabel('True Label')

# Show the plot
plt.show()
